**WILDFIRES UPDATE 2022 APPROACH - AP2022**

## General overview

Process overview to update the burnt areas dataset:
- Data extracion from the JRC EFFIS/Wildfire Database as a shapefile
- Filter per year (field name 'YEAR')
- Rasterization (one separate raster per year)
- Summing all the wildfires to compute wildfires recurrence

## Data considerations

**IMPORTANT NOTE:**
The Rapid Damage Assessment (RDA) module of EFFIS was initially implemented in 2003 to map burned areas during the fire season,
by analyzing MODIS daily images at 250 m spatial resolution. For the monitoring of burned areas during the summer,
daily images from the MODIS instruments on board of TERRA and AQUA satellites are acquired and processed few hours after the acquisition.

The EFFIS Rapid Damage Assessment provides the daily update of the perimeters of burnt areas in Europe twice, every day **for fires of about 30 ha or larger**.

For any use of the EFFIS Burnt Area product the conditions listed below must be taken into consideration:
 - The product is derived from the daily processing of MODIS satellite imagery at 250 m ground spatial resolution.
 - The perimeters of the burned scars in the represent areas burned by fires as detected from MODIS satellite imagery. 
   Therefore, no distinction is made between wildland fires, environmental burnings or prescribed fires.
 - Burnt scars of approximately 30 hectares in size are mapped, although the product may also include the perimeters
   of burned areas of smaller dimension.
 - Small burnt or un-burnt areas below the spatial resolution of the MODIS imagery are not mapped; 
   these may include small unburned islands inside the burnt area perimeter.
 - The Burnt Area product is updated up two times every day. 
   The perimeter of burnt areas due to different fires happening between two sequential updates may be merged into a single perimeter.
 - The dates reported as Start date and Last update may not correspond to the date of ignition and extinction of the fire
 - The EFFIS burnt area product is intended to provide estimates of burnt areas at European level. Caution should be taken when comparing this product to other data that may have been produced using different methodologies and scopes.
 - Although the burnt area perimeters are overlayed on the Google imagery, it must be noted that the original EFFIS product 
   is produced at 250 m spatial resolution, not at the resolution of the underlying Google images.

Although only a fraction of the total number of fires is mapped, the area burned by fires of this size represents 
**about 75% to 80%** of the total area burned in EU.

Modeling historical fire data from the EFFIS European fire database, equations have been set up for the different countries, 
which allow predicting the total area burned with good accuracy; that is, the total burnt area in a country is obtained as a
function of the area burned by fires larger than 30 ha.

<hr>

## Process of updating the burnt areas dataset

### Importing modules and settting variables

In [1]:
# -*- coding: utf-8 -*-
# ------------------------------------------------------------------------------------------
# Created on: 2022-07-11
# Description: Adaption of processing to incorporate most recent years data for burnt areas
# Author: Raquel Ubach (raquel.ubach@uab.cat)
# ------------------------------------------------------------------------------------------

# Import arcpy module
import arcpy, os
from arcpy.sa import *
import pandas as pd
import numpy as np

# Setting workspace folders and variables
#arcpy.env.extent = "900000 900000 7400000 5500000"
in_folder = r"S:\\Common workspace\\ETCULS\\Vulnerability\\AP2022"
ba_00_22 =  in_folder + "\\InputData\\effis_layer_00_22\\ba_00_22.shp" #input features

out_ws = in_folder + "\\Output_22" #folder to store results
if not os.path.exists(out_ws): #create folder if it does not exist
    os.makedirs(out_ws)

# CreateFileGDB to store intemediate files
out_gdb = "BurntAreas_22.gdb"
out_gdb_path  = out_ws + "\\" + out_gdb
if not arcpy.Exists(out_gdb_path):
    arcpy.CreateFileGDB_management(out_ws, out_gdb)

### Preparing data: adding required fields

In [2]:
# Adding field to store effis id (as a numeric field) 
fieldname = "EFFISID"
arcpy.AddField_management(ba_00_22, fieldname, "LONG")
expression = 'int(!id!.rstrip())'
arcpy.CalculateField_management(ba_00_22, fieldname, expression, "PYTHON3")

<Result 'S:\\\\Common workspace\\\\ETCULS\\\\Vulnerability\\\\AP2022\\InputData\\ba_00_22.shp'>

In [55]:
# Adding field to store year 
fieldname = "YEAR"
arcpy.AddField_management(ba_00_22, fieldname, "LONG", 4, "", "", "year", "NULLABLE", "REQUIRED")
expression = '!FIREDATE!.split("-")[0]'
arcpy.CalculateField_management(ba_00_22, fieldname, expression, "PYTHON3")

<Result 'S:\\\\Common workspace\\\\ETCULS\\\\Vulnerability\\\\AP2022\\InputData\\ba_00_22.shp'>

In [40]:
# Adding field to store month
fieldname = "MONTH"
arcpy.AddField_management(ba_00_22, fieldname, "LONG", 2)
expression = '!FIREDATE!.split("-")[1]'
arcpy.CalculateField_management(ba_00_22, fieldname, expression, "PYTHON3")

<Result 'S:\\\\Common workspace\\\\ETCULS\\\\Vulnerability\\\\AP2022\\InputData\\ba_00_22.shp'>

In [43]:
# Adding field to store day
fieldname = "DAY"
arcpy.AddField_management(ba_00_22, fieldname, "LONG", 2)
expression = '!FIREDATE!.split("-")[2][:2]'
arcpy.CalculateField_management(ba_00_22, fieldname, expression, "PYTHON3")

<Result 'S:\\\\Common workspace\\\\ETCULS\\\\Vulnerability\\\\AP2022\\InputData\\ba_00_22.shp'>

In [12]:
# Adding field to store country code
fieldname = "Ctry"
arcpy.AddField_management(ba_00_22, fieldname, "TEXT", field_length=2)
expression = '!COUNTRY!.rstrip()'
arcpy.CalculateField_management(ba_00_22, fieldname, expression, "PYTHON3")

<Result 'S:\\\\Common workspace\\\\ETCULS\\\\Vulnerability\\\\AP2022\\InputData\\ba_00_22.shp'>

In [13]:
# Join field to allow filtering by EEA38, EA27 
in_tbl = in_folder + "\\InputData\\Country_code.csv"
csv = pd.read_csv(in_tbl, sep=',')
#tbl = arcpy.conversion.TableToTable(csv, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})
inFeatures=ba_00_22
joinField='Ctry'
joinTable=in_tbl
joinField='Ctry'
arcpy.JoinField_management(inFeatures, joinField, joinTable, joinField)

<Result 'S:\\\\Common workspace\\\\ETCULS\\\\Vulnerability\\\\AP2022\\InputData\\ba_00_22.shp'>

### Accessing data and processing by year

In [5]:
arcpy.env.overwriteOutput = True
#arcpy.env.extent = "900000 900000 7400000 5500000"
arcpy.env.extent = "1339500 666300 7579500 5326300"

years = [*range (2000, 2023, 1)]

for i in years:
    #Select events per year
    layer_file = "layer_" + str(i)
    where_clause = "YEAR = " + str(i)
    if not arcpy.Exists(layer_file):
        arcpy.management.MakeFeatureLayer(ba_00_22, layer_file, where_clause)
    
    #Rasterize input elements
    outRaster = out_gdb_path + "\\BA_" + str(i) + "_r"
    if not arcpy.Exists(outRaster):
        arcpy.FeatureToRaster_conversion(layer_file, "EFFISID", outRaster, 100)
  
    #Export to tif
    tif = out_ws + "\\BA" + str(i) + ".tif"
    if not arcpy.Exists(tif):
        arcpy.CopyRaster_management(outRaster, tif, "", "", "0", "NONE", "NONE", "32_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE")

### Creating Look-Up Table (LUT)

In [2]:
from arcgis.gis import GIS
df = pd.DataFrame.spatial.from_featureclass(ba_00_22)

In [12]:
col_ls = df.columns.values.tolist()
col_ls

['FID', 'id', 'FIREDATE', 'LASTUPDATE', 'COUNTRY', 'PROVINCE', 'COMMUNE', 'AREA_HA', 'BROADLEA', 'CONIFER', 'MIXED', 'SCLEROPH', 'TRANSIT', 'OTHERNATLC', 'AGRIAREAS', 'ARTIFSURF', 'OTHERLC', 'PERCNA2K', 'CLASS', 'YEARSEASON', 'Ctry', 'Country_1', 'EU27', 'EEA38', 'YEAR', 'MONTH', 'DAY', 'EFFISID', 'SHAPE']

In [3]:
sel_cols = ['EFFISID', 'AREA_HA', 'COMMUNE', 'PROVINCE', 'COUNTRY', 'Country_1', 'EU27', 'EEA38', 'FIREDATE', 'LASTUPDATE', 'YEAR', 'MONTH', 'DAY']
df2 = df.loc[:, sel_cols]
df2.head()

,EFFISID,AREA_HA,COMMUNE,PROVINCE,COUNTRY,Country_1,EU27,EEA38,FIREDATE,LASTUPDATE,YEAR,MONTH,DAY
0,27361,95,San Giovanni Rotondo,Foggia,IT,Italy,1,1,2000-01-01 00:00:00,2000-01-01 00:00:00,2000,1,1
1,27785,91,"Cepões, Meijinhos e Melcões",Douro,PT,Portugal,1,1,2000-01-01 00:00:00,2000-01-01 00:00:00,2000,1,1
2,27809,126,Ragusa,Ragusa,IT,Italy,1,1,2000-01-01 00:00:00,2000-01-01 00:00:00,2000,1,1
3,27827,432,Avessadas e Rosém,Tâmega e Sousa,PT,Portugal,1,1,2000-01-01 00:00:00,2000-01-01 00:00:00,2000,1,1
4,27865,70,Paranhos,Beiras e Serra da Estrela,PT,Portugal,1,1,2000-01-01 00:00:00,2000-01-01 00:00:00,2000,1,1


In [19]:
# Export to csv table
out_tbl = out_ws + '\\LUT_EFFIS_Fires_00_22.csv'
if not os.path.exists(out_tbl):
    df2.to_csv(out_tbl, encoding='utf-8')